In [1]:
import pandas as pd
import numpy as np

## Portfolio

- AEX: €300 000
- SPY: €300 000
- HSI: €90 000
- RDSA: €10 000
- Bonds: €300 000

Total: €1 000 000

In [116]:
data = pd.read_csv('dataset.csv', index_col=0)
data.index = pd.to_datetime(data.index)

In [132]:
stock_returns = np.log(data[['AEX', 'SPY', 'RDSA.AS', 'HSI']]).diff()
stock_returns[['SPY', 'HSI']] + np.log(data[['USDEUR', 'HKDEUR']]).diff().values
yields = data['YIELDS']